In [1]:
import numpy as np
import cvxopt
from sklearn.model_selection import train_test_split

var = 0.1
n = 250

x00 = np.hstack((np.random.normal(0,var,(n,1)) , np.random.normal(0,var,(n,1)) ))
x01 = np.hstack((np.random.normal(0,var,(n,1)) , np.random.normal(1,var,(n,1)) ))
x10 = np.hstack((np.random.normal(1,var,(n,1)) , np.random.normal(0,var,(n,1)) ))
x11 = np.hstack((np.random.normal(1,var,(n,1)) , np.random.normal(1,var,(n,1)) ))
x = np.asarray(np.concatenate((x00,x01,x10,x11)))
print(x.shape)
y = np.asarray(np.concatenate((-1 * np.ones((n,1)),np.ones((2*n,1)),(-1 * np.ones((n,1)))))).reshape(-1)

data = x
labels = y
labels = labels.astype(np.double)
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, stratify=labels)

# print(data_train[0:10])
# print(labels_train[0:10])

(1000, 2)


In [2]:
def rbf_kernel(x,y, gamma = 0.1):
        return np.exp(-1.0*gamma*np.dot(np.subtract(x,y).T,np.subtract(x,y)))

In [3]:
data_RBF = np.zeros((len(data_train),len(data_train)))
gamma = 0.1
for i in range(len(data_train)):
    for j in range(len(data_train)):
        temp = -np.linalg.norm(data_train[i]-data_train[j])**2
        data_RBF[i][j] = np.exp(temp*gamma)

# data_RBF = cp.Constant(data_RBF)
print(data_RBF.shape)

(800, 800)


In [4]:
n_samples = len(labels_train)
temp = np.outer(labels_train,labels_train)
print(temp.shape)
P = cvxopt.matrix(temp* data_RBF)
q = cvxopt.matrix(np.ones(n_samples) * -1.0)
A = cvxopt.matrix(labels_train, (1,n_samples))
b = cvxopt.matrix(0.0)
G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
h = cvxopt.matrix(np.zeros(n_samples))

(800, 800)


In [5]:
print(np.array(P).shape, np.array(q).shape, np.array(A).shape, np.array(b).shape, np.array(G).shape, np.array(h).shape)
solution = cvxopt.solvers.qp(P, q, G, h, A, b)

(800, 800) (800, 1) (1, 800) (1, 1) (800, 800) (800, 1)
     pcost       dcost       gap    pres   dres
 0: -2.2481e+02 -6.3037e+02  2e+03  4e+01  2e+00
 1: -3.1582e+02 -7.0486e+02  7e+02  1e+01  6e-01
 2: -4.6208e+02 -8.4033e+02  7e+02  1e+01  5e-01
 3: -9.2679e+02 -1.3033e+03  7e+02  1e+01  5e-01
 4: -1.4208e+03 -1.7065e+03  4e+02  4e+00  2e-01
 5: -1.4772e+03 -1.5092e+03  5e+01  4e-01  2e-02
 6: -1.4821e+03 -1.4842e+03  2e+00  4e-03  2e-04
 7: -1.4838e+03 -1.4838e+03  2e-02  4e-05  2e-06
 8: -1.4838e+03 -1.4838e+03  2e-04  4e-07  2e-08
 9: -1.4838e+03 -1.4838e+03  2e-06  4e-09  2e-10
Optimal solution found.


In [6]:
a = np.ravel(solution['x'])
print(a.shape)

S = a>1e-5

(800,)


In [7]:
SVs = data_train[S]
n_support = np.sum(S)
a = a[a>1e-5]
SL = labels_train[S]
indices = np.arange(n_samples)[S]
intercept = 0
for i in range(a.shape[0]):
    intercept += SL[i] 
    intercept -= np.sum(a*SL*data_RBF[indices[i],S])
intercept /= a.shape[0]

In [8]:
score = np.zeros(data_test.shape[0])
for i in range(data_test.shape[0]):
    s = 0
    for alpha,label,sv in zip(a,SL,SVs):
        s += alpha*label*rbf_kernel(data_test[i],sv)
    score[i] = s
score = score + intercept

res = np.where(score>0,1,-1)
print(res)

[-1  1  1 -1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1  1 -1
  1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1  1  1 -1  1  1  1  1 -1 -1 -1
 -1  1  1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1  1  1  1 -1  1  1  1  1 -1
  1  1 -1  1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1  1  1 -1  1  1  1  1  1
  1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1  1  1  1  1  1
  1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1  1 -1 -1  1
  1  1  1 -1  1  1 -1  1 -1 -1  1 -1  1  1  1 -1  1 -1 -1  1  1 -1 -1 -1
 -1 -1  1 -1  1  1  1 -1  1 -1 -1  1  1 -1  1 -1  1 -1 -1  1  1  1 -1  1
  1  1 -1  1  1  1 -1 -1]


In [9]:
print(labels_test.flatten())

[-1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1.  1.  1.  1.
 -1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
 -1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1.
 -1.  1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.
  1.  1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1. -1.  1.  1.
 -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1.
  1. -1. -1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1.
 -1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.
  1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1.  1. -1.  1. -1.
 -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.
  1. -1.  1. -1.  1. -1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.
 -1. -1.]


In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(res, labels_test.flatten())

1.0

In [13]:
import numpy as np
import cvxpy as cp
import pandas as pd

X = data_train
Y = labels_train

# Convex Optimization
a = cp.Variable(len(Y))
R1 = cp.matmul(cp.diag(a),Y)
R2 = cp.matmul(X.T,R1)
R4 = cp.norm(R2)**2
R4.shape

P1 = cp.sum(a)
Const1 = P1 - 0.5*R4
# Const1 = np.reshape(Const1,(1,))

Const2 = cp.matmul(a.T,Y)
Const3 = [0<=a,Const2 == 0]
obj = cp.Maximize(Const1)
prob = cp.Problem(obj, Const3)
prob.solve(verbose=True)

W = np.zeros((2,))
# for i in range(len(Y)):
#     W += A[i]*Y[i]*(X[i].T)
#     if(A[i]>1e-4):
# #         print(i)
A = (np.array(a.value)).reshape(-1,1)
W0 = (1/Y[327]) - np.dot(W,X[327])
# print(W0)

Test = data_test
preds_lin = []
for i in range(len(Test)):
    est = np.sign(np.dot(W,Test[i])+W0)
    preds_lin.append(est)
    
linals = accuracy_score(preds_lin, labels_test)
print(linals)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -5.970e-01  +9e+02  1e+00  1e+00  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -2.809e+02  -9.000e+01  +4e+02  9e+01  5e+01  3e+02  6e-01  0.5588  1e-01   1  1  1 |  0  0
 2  -3.860e+02  -4.012e-01  +3e+02  7e-01  1e+00  4e+02  5e-01  0.0412  1e+00   1  1  1 |  0  0
 3  -3.230e+03  -3.380e+00  +7e+01  1e+00  2e+00  3e+03  1e-01  0.7735  3e-02   1  1  1 |  0  0
 4  -4.276e+03  -2.558e+00  +7e+01  1e+00  3e+00  4e+03  1e-01  0.3027  7e-01   2  2  2 |  0  0
 5  -1.779e+04  -1.290e+01  +2e+01  1e+00  3e+00  2e+04  3e-02  0.7395  2e-02   2  1  1 |  0  0
 6  -3.594e+05  -3.358e+02  +1e+00  2e+00  4e+00  4e+05  2e-03  0.9504  2e-02   1  1  1 |  0  0
 7  -3.107e+07  -5.863e+04  +1e-01  1e+01  2e+01  3e+07  2e-04  0.9890  7e-02   1  1  1 |  0  0
 8  -4.300e+08  -7.059e+05  +1e-02  1e+01  3e+

As we can observe, RBF kernel outperform linear kernel as XOR is a non linear function by nature